# The Inc operator:
## Direct problem in 3D

The problem is :given $\gamma$, find $f$ such that:
$$
\begin{cases}
\text{inc } \gamma = f & \text{in } \Omega\\
\left . f \right \vert_{\partial\Omega} = f_0 & \text{in } \partial\Omega
\end{cases}
$$

In the thesis is proven that the distributive $\text{inc}$ operator is given by:
$$
    \left\langle \text{ inc }\gamma , \phi \right\rangle
    =
     \sum_T\left\lbrace \int_T \text{ inc } \gamma:  \phi  
    -\sum_{F\subset T} \left[\int_{F}\left[\text{ curl } \gamma\right]_{QQ}: \left[C_n\phi\right]_{QQ} 
    - \text{ rot } _F ( \left[C_n\gamma\right]_{PQ}) : \left[ \phi\right]_{QQ}
    -\sum_{E \subset F}\int_{E}
    \left[C_n\gamma \right]_{PQ} n  \cdot \left[\phi\right]_{QQ} t_{E}\right] \right\rbrace 
$$
with 
$$ rot_F( u ) = curl(u P_n)Q_n $$
If the following conditions are satisfied:

1. $\text{inc }\gamma_i$ is well defined for all $\gamma_i$  function on the triangle  $T_i$ .
2. $\gamma := \sum_i \gamma_i$ is tt-continuous on the boundary of $T_i$.
3. All the above integrals are well define on the faces and eges of the triangulation.

$$


In [16]:
import time

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
from ngsolve.krylovspace import CGSolver

In [17]:
n = specialcf.normal(3) # normal vector to a surface of a tetrahedron

def norm(u, Mesh):
    # norm of a scalar, vector or matrix function
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh)) 

def Curl(u):
    # curl of a vector or matrix
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )

def Inc(u):
    # inc of a matrix
    return Curl((Curl(u)).trans)

P_n = OuterProduct(n,n)  # projection along normal direction
Q_n = Id(3) - OuterProduct(n,n) # projection along tangential direction
C_n = CF( (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) ) # corss product perator with the normal vector

def C(V): 
    # corss product perator with the generic vector
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 


In [20]:
def distributive_inc(mesh,order = 1, gamma = None, BND_Inc_gamma = None ,draw = True, bonus_intorder = 15):
    
    fesHCurlCurl = HCurlCurl(mesh, order=order, dirichlet= ".*")     
    
    gf_gamma = GridFunction(fesHCurlCurl) 
    gf_gamma.Set ( gamma, bonus_intorder=10, dual=True)

    gf_inc_gamma = GridFunction(fesHCurlCurl)
    gf_inc_gamma.Set(BND_Inc_gamma, BND,  bonus_intorder=9, dual=True)

    ##########################################################
    #              Linear and Bilinear forms                 #
    ##########################################################

    u,v = fesHCurlCurl.TnT()

    #some geometrical objects we need

    n = specialcf.normal(3) # normal vector to a surface of a tetrahedron
    n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
    t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
    t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
    e = specialcf.tangential(3,True)
    n1 = Cross( t1, e)
    n2 = Cross( t2, e) 

    # Mass matrix
    a = BilinearForm(fesHCurlCurl)
    a += InnerProduct(u,v)*dx 

    # linear form induced by the metric gfG
    f = LinearForm(fesHCurlCurl)

    # thetrahedron inc part
    f += InnerProduct(gf_gamma.Operator("inc"),v)*dx        
    # faces part:
    f += ( InnerProduct(Q_n*n_cross_v, curl(gf_gamma).trans) + Cross(gf_gamma*n,n)*(curl(v)*n) )*dx(element_boundary=True)
    # Edges components: t'*v*C_n*n
    f += (gf_gamma[n1,e]*v[e,t1] - gf_gamma[n2,e]*v[e,t2])*dx(element_vb=BBND)
        


    ##########################################################
    #           Assemble, compute and print!                 #
    ##########################################################

    pre = Preconditioner(a, "local")
    a.Assemble()
    f.Assemble()

    r = f.vec.CreateVector()
    r.data = f.vec - a.mat * gf_inc_gamma.vec
    inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-12)
    gf_inc_gamma.vec.data += inverse * r
    #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r
        

    if draw:
        print("gf_gamma")
        Draw(gf_gamma, mesh, "gf_gamma", draw_surf=True, clipping=(0,1,1), deformation=False)
        print("gamma")
        Draw(gamma, mesh, "gf_gamma", draw_surf=True, clipping=(0,1,1), deformation=False)
        print("norm(gf_gamma- gamma) = ", norm(gamma-gf_gamma,mesh))
        Draw(gf_gamma - gamma, mesh, "gf_gamma - gamma", draw_surf=True, clipping=(0,1,1), deformation=False)

        print("gf_inc_gamma")
        Draw(gf_inc_gamma, mesh, "gf_inc_gamma", draw_surf=True, clipping=(0,1,1), deformation=False)
        print("inc(gamma)")
        Draw(BND_Inc_gamma, mesh, "inc(gamma)", draw_surf=True, clipping=(0,1,1), deformation=False)
        print("norm(gf_inc_gamma - inc(gamma)) = ", norm(gf_inc_gamma - BND_Inc_gamma, mesh))
        Draw(gf_inc_gamma - BND_Inc_gamma, mesh, "gf_inc_gamma - BND_Inc_gamma", draw_surf=True, clipping=(0,1,1), deformation=False)
        
    return gf_inc_gamma , gf_gamma 

In [23]:
#mesh a cube
maxh = 0.10
mesh = Mesh(unit_cube.GenerateMesh(maxh=maxh))

# order of Regge FE
order = 2

# functions
peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))
cf_gamma = CF ( (peak, 0, 0 , 0,peak,0,0,0,peak), dims=(3,3))
cf_inc_gamma = CF( Inc(cf_gamma), dims=(3,3) )

# flag for drawing
draw= True 

# what intorder we want..
bonus_intorder = 15


with TaskManager():
    gf_inc_gamma , gf_gamma = distributive_inc(mesh,order = order, gamma = cf_gamma, BND_Inc_gamma = cf_inc_gamma ,draw = draw, bonus_intorder = bonus_intorder)
 

CG NOT converged in 1000 iterations to residual 2.461934491115237e-10
gf_gamma


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

gamma


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

norm(gf_gamma- gamma) =  0.0009271478137042044


...

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

inc(gamma)


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

norm(gf_inc_gamma - inc(gamma)) =  0.0984039820635727


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…